In [3]:
import os
from datetime import datetime

import pandas as pd
from notion_client import Client

In [5]:
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Union


@dataclass
class ColumnDefaultConfig:
    """
    Default configuration for columns.
    This class is used to define the default settings for columns in the domain configuration.
    """

    """ Defines the column name for the configuration."""
    columnName: str = None


@dataclass
class RawSheetConfig(ColumnDefaultConfig):
    pass


@dataclass
class InvoicePdfConfig(ColumnDefaultConfig):
    index: Optional[int] = None
    heading: Optional[bool] = None


@dataclass
class DatabaseConfig(ColumnDefaultConfig):
    columnName: str = None


@dataclass
class ColumnConfig(ColumnDefaultConfig):
    invoicePdf: Optional[InvoicePdfConfig] = None
    rawSheet: Optional[RawSheetConfig] = None
    database: Optional[DatabaseConfig] = None


@dataclass
class Location:
    locationName: str
    shippingAddress: str
    retailer: str
    code: str = None
    storeId: str = None


@dataclass
class DomainConfig:
    databaseId: str
    databaseLocationId: str
    columns: List[ColumnConfig] = field(default_factory=list)
    locations: List[Location] = field(default_factory=dict)


domainConfig: DomainConfig = DomainConfig(
    columns=[
        ColumnConfig(
            columnName="Article Code",
            invoicePdf=InvoicePdfConfig(columnName="Article Code", index=1),
            rawSheet=RawSheetConfig(columnName="ITEM_CODE"),
            database=DatabaseConfig(columnName="Article Code"),
        ),
        ColumnConfig(
            columnName="Date",
            invoicePdf=InvoicePdfConfig(columnName="Date", heading=True),
            rawSheet=RawSheetConfig(columnName="Date"),
            database=DatabaseConfig(columnName="Date"),
        ),
        ColumnConfig(
            columnName="Dispatched Qty",
            invoicePdf=InvoicePdfConfig(columnName="Dispatched Qty", index=4),
            rawSheet=RawSheetConfig(columnName="Indents"),
            database=DatabaseConfig(columnName="Dispatched Qty"),
        ),
        ColumnConfig(
            columnName="Item Description",
            invoicePdf=InvoicePdfConfig(columnName="Item Description", index=2),
            rawSheet=RawSheetConfig(columnName="PRODUCT_NAME"),
            database=DatabaseConfig(columnName="Item Description"),
        ),
        ColumnConfig(
            columnName="Invoice No",
            invoicePdf=InvoicePdfConfig(columnName="Invoice No", heading=True),
            database=DatabaseConfig(columnName="Invoice No"),
        ),
        ColumnConfig(
            columnName="Invoice Version",
            database=DatabaseConfig(columnName="Invoice Version"),
        ),
        ColumnConfig(
            columnName="Location",
            invoicePdf=InvoicePdfConfig(columnName="Location", heading=True),
            rawSheet=RawSheetConfig(columnName="STORE_NAME"),
            database=DatabaseConfig(columnName="Location"),
        ),
        ColumnConfig(
            columnName="PO No",
            invoicePdf=InvoicePdfConfig(columnName="PO No", heading=True),
            rawSheet=RawSheetConfig(columnName="PO Number"),
            database=DatabaseConfig(columnName="PO No"),
        ),
        ColumnConfig(
            columnName="Rate",
            invoicePdf=InvoicePdfConfig(columnName="Rate", index=6),
            rawSheet=RawSheetConfig(columnName="Cost"),
            database=DatabaseConfig(columnName="Rate"),
        ),
        ColumnConfig(
            columnName="Recieved Qty",
            invoicePdf=InvoicePdfConfig(columnName="Recieved Qty", index=5),
        ),
        ColumnConfig(
            columnName="Retailer",
            invoicePdf=InvoicePdfConfig(columnName="Retailer", heading=True),
            rawSheet=RawSheetConfig(columnName="Entity Name"),
            database=DatabaseConfig(columnName="Retailer"),
        ),
        ColumnConfig(
            columnName="Sr", invoicePdf=InvoicePdfConfig(columnName="Sr", index=0)
        ),
        ColumnConfig(
            columnName="Total Amount",
            invoicePdf=InvoicePdfConfig(columnName="Total Amount", index=7),
            # rawSheet=RawSheetConfig(columnName="Total"),
            database=DatabaseConfig(columnName="Total Amount"),
        ),
        ColumnConfig(
            columnName="UoM",
            invoicePdf=InvoicePdfConfig(columnName="UoM", index=3),
            rawSheet=RawSheetConfig(columnName="WEIGHT"),
            database=DatabaseConfig(columnName="UoM"),
        ),
        ColumnConfig(
            columnName="Vendor Name",
            invoicePdf=InvoicePdfConfig(columnName="Vendor Name", heading=True),
            rawSheet=RawSheetConfig(columnName="VENDOR"),
            database=DatabaseConfig(columnName="Vendor Name"),
        ),
    ],
    locations=[
        Location(
            locationName="Ayodhya Nagar",
            shippingAddress="Gadewar Lawns Plot No.31, 32, 33, 36, 37 And 38, K. H. No, 72/2, Situated At Gadewar Lawn, Shri Ram Wadi",
            retailer="Rajidi Retail Pvt Ltd",
            code="AN",
            storeId="1403419",
        ),
        Location(
            locationName="Byramji",
            shippingAddress="Unit nos - 59 to 71 Lower Ground Floor Ginger Square City Survey No - 1049",
            retailer="Rajidi Retail Pvt Ltd",
            code="B",
            storeId="1392084",
        ),
        Location(
            locationName="Dharampeth",
            shippingAddress="Plot No. 151, CTS No. 135 Puja Sabhagrah, Ravi Nagar Square, Ram Nagar",
            retailer="Swinsta Ent Private Limited",
            code="DH",
            storeId="1397624",
        ),
        Location(
            locationName="Mahal",
            shippingAddress="Unit no - G-1, Plot no.58, sardar patel timber Dhantoli, NAGPUR - 440027",
            retailer="Rajidi Retail Pvt Ltd",
            code="MH",
            storeId="1393571",
        ),
        Location(
            locationName="Manish Nagar",
            shippingAddress='Ground floor "Jayanti Mansion III", Manish nagar Nagpur Maharashtra',
            retailer="Rajidi Retail Pvt Ltd",
            code="MN",
            storeId="1392532",
        ),
        Location(
            locationName="Nandanvan",
            shippingAddress="Vinayak Tower, Lower Ground Floor, Survey No.212 Gurudev Nagar Main Road, New Nanadanvan",
            retailer="Swinsta Ent Private Limited",
            code="NA",
            storeId="1397035",
        ),
        Location(
            locationName="Sai Mandir",
            shippingAddress="Khasra No 18/2, city Survey No.718, House No. 781/B, Situated at Village Ajni",
            retailer="Swinsta Ent Private Limited",
            code="S",
            storeId="1399707",
        ),
    ],
    databaseId="23445ba7974f8081852acc595fd9436c",
    databaseLocationId="d4b0f1c2e3f14a8",
)

# to go

In [11]:
notion = Client(auth="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")

responseDf = pd.DataFrame(
    columns=[
        columnName.database.columnName
        for columnName in domainConfig.columns
        if columnName.database is not None
    ],
)

startCursor = None

In [6]:
[
    columnName.database.columnName
    for columnName in domainConfig.columns
    if columnName.database is not None
]

['Article Code',
 'Date',
 'Dispatched Qty',
 'Item Description',
 'Invoice No',
 'Invoice Version',
 'Location',
 'PO No',
 'Rate',
 'Retailer',
 'Total Amount',
 'UoM',
 'Vendor Name']

In [13]:
def notionObject2DataFrame(notionObject):
    data = {}
    for key, value in notionObject["properties"].items():
        if value["type"] == "title":
            data[key] = value["title"][0]["plain_text"]
        elif value["type"] == "select":
            data[key] = value["select"]["name"]
        elif value["type"] == "number":
            data[key] = value["number"]
        elif value["type"] == "date":
            data[key] = value["date"]["start"]
        elif value["type"] == "relation":
            # data[key] = notion.pages.retrieve(value["relation"][0]["id"])
            # print data[key]
            data[key] = [relation["id"] for relation in value["relation"]]
        elif value["type"] == "formula":
            if "string" in value["formula"]:
                data[key] = value["formula"]["string"]
            elif "number" in value["formula"]:
                data[key] = value["formula"]["number"]
    return data


databaseColumns = [
    "Name",
    "Map",
    # "Article Code",
    # "Item Description",
    # "UoM",
    # "Dispatched Qty",
    # "Rate",
    # "Total Amount",
    # "Date",
    # "Location",
    # "Invoice No",
    # "Invoice Version",
]

In [16]:
while True:
    response = notion.databases.query(
        database_id="24b3d0999ee8809abebfc511c513b0ad",
        # database_id="24845ba7974f808a94e9c967f4113269",
        filter={"property": "Date", "date": {"equals": "2025-08-06"}},
        start_cursor=startCursor,
    )
    for result in response["results"]:
        responseDf = pd.concat(
            [responseDf, pd.DataFrame([notionObject2DataFrame(result)])],
            ignore_index=True,
        )
        print(f"Fetched {len(response['results'])} results")
    if response.get("has_more"):
        startCursor = response["next_cursor"]
    else:
        break

APIResponseError: The start_cursor provided is invalid: 24b3d099-9ee8-817f-a595-c84bbd0c3716

In [24]:
# response["results"]
responseDf

,Name,Map
0,Major Lazer,"{'object': 'page', 'id': '24945ba7-974f-80e3-9..."
1,Lean On,"{'object': 'page', 'id': '24945ba7-974f-8002-a..."
2,Major Lazer,[24945ba7-974f-80e3-95ae-dc9848bb90ef]
3,Lean On,[24945ba7-974f-8002-ad47-da5fca0cb4af]


In [ ]:
pageId = "24945ba7974f8043b422dac622593f52"
result = notionObject2DataFrame(
    notion.pages.retrieve(
        page_id=pageId,
        # properties={
        #     "Name": {"title": [{"text": {"content": "Updated Name"}}]},
        #     "Map": {"relation": [{"text": {"content": "Updated Map"}}]},
        # },
    )
)

In [18]:
result

,Code,Store Id,Retailer,Location
0,AN,1403419,Rajidi,Ayodhya Nagar


In [16]:
df1 = pd.DataFrame(
    {
        "Article Code": ["A001", "A002", "A003", "A005"],
        "Item Description": ["Item 1", "Item 2", "Item 3", "Item 5"],
        "UoM": ["kg", "kg", "kg", "kg"],
    }
)
df2 = pd.DataFrame(
    {
        "Article Code": ["A001", "A002", "A003", "A004"],
        "Item Description": ["Item 1", "Item 2", "Item 3", "Item 4"],
        "UoM": ["kg", "kg", "kg", "kg"],
    }
)

In [21]:
mrg = df1.merge(df2, how="right", indicator=True)
mrg[mrg["_merge"] == "right_only"].drop(columns=["_merge"])

,Article Code,Item Description,UoM
3,A004,Item 4,kg
